In this notebook, we introduce how to train VAE to detect fraud
### Load Data

In [1]:
import openmldb
db = openmldb.dbapi.connect(zk='127.0.0.1:2181',zkPath='/openmldb')
openmldb.sql_magic.register(db)

2023-07-05 13:06:38,747:3996(0x7f8be647c740):ZOO_INFO@log_env@753: Client environment:zookeeper.version=zookeeper C client 3.4.14
2023-07-05 13:06:38,747:3996(0x7f8be647c740):ZOO_INFO@log_env@757: Client environment:host.name=cdd2a59992ee
2023-07-05 13:06:38,747:3996(0x7f8be647c740):ZOO_INFO@log_env@764: Client environment:os.name=Linux
2023-07-05 13:06:38,747:3996(0x7f8be647c740):ZOO_INFO@log_env@765: Client environment:os.arch=5.10.16.3-microsoft-standard-WSL2
2023-07-05 13:06:38,747:3996(0x7f8be647c740):ZOO_INFO@log_env@766: Client environment:os.version=#1 SMP Fri Apr 2 22:23:49 UTC 2021
2023-07-05 13:06:38,747:3996(0x7f8be647c740):ZOO_INFO@log_env@774: Client environment:user.name=(null)
2023-07-05 13:06:38,747:3996(0x7f8be647c740):ZOO_INFO@log_env@782: Client environment:user.home=/root
2023-07-05 13:06:38,747:3996(0x7f8be647c740):ZOO_INFO@log_env@794: Client environment:user.dir=/work/adops
2023-07-05 13:06:38,747:3996(0x7f8be647c740):ZOO_INFO@zookeeper_init@827: Initiating clie

In [2]:
import utils
train_feature_dir='/tmp/train_feature'
datadf = utils.read_dataset(f'{train_feature_dir}/*.csv')
print(datadf.head(10))

   is_attribute  app  device  os  channel  hour  day  qty  ip_app_count  \
0             0    9       1  13      244     0    8    1             1   
1             0   11       1  22      319     9    7    1             1   
2             0   12       1  19      140    15    7    1             1   
3             0   18       1  13      107    19    8    1             1   
4             0    3       1  23      417    19    7    1             1   
5             0   12       1  20      259    18    8    1             1   
6             0    3       1  19      182    15    9    1             1   
7             0   21       1  19      128    16    9    1             1   
8             0    7       1  13      101    23    9    1             1   
9             0    3       1  20      280     9    9    1             1   

   ip_app_os_count  
0                1  
1                1  
2                1  
3                1  
4                1  
5                1  
6                1  
7     

/usr/local/lib/python3.9/dist-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
from keras.models import Model, load_model
from keras.layers import Input, Dense,LeakyReLU,BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras import regularizers

2023-07-05 13:06:39.512505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-05 13:06:39.512547: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
X = datadf.drop(['is_attribute'], axis=1)
Y = datadf.is_attribute
input_dim    = X.shape[1]
encoding_dim = 128
num_epoch    = 30
batch_size   = 256

In [6]:
input_layer = Input(shape=(input_dim, ))

encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder =BatchNormalization()(encoder)
encoder=LeakyReLU(alpha=0.2)(encoder)

encoder = Dense(int(encoding_dim/2), activation="relu")(encoder)
encoder =BatchNormalization()(encoder)
encoder=LeakyReLU(alpha=0.1)(encoder)

encoder = Dense(int(encoding_dim/4), activation="relu")(encoder)
encoder =BatchNormalization()(encoder)



### decoder
decoder = LeakyReLU(alpha=0.1)(encoder)
decoder = Dense(int(encoding_dim/4),activation='tanh')(decoder)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU(alpha=0.1)(decoder)


decoder = Dense(int(encoding_dim/2), activation='tanh')(decoder)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU(alpha=0.1)(decoder)

decoder = Dense(input_dim)(decoder)

autoencoder = Model(inputs = input_layer, outputs = decoder)
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae','mse'])

checkpointer = ModelCheckpoint(filepath="autoencoder.h5", verbose=0, save_best_only=True)
history = autoencoder.fit(X, 
                          X,
                          epochs=num_epoch,
                          batch_size=batch_size,
                          shuffle=True,
                          #validation_data=(X_test, X_test),
                          verbose=1, 
                          callbacks=[checkpointer]
                          ).history
autoencoder.save("/work/adops/saved_models/ae.h5")

Epoch 1/30


2023-07-05 13:07:17.988021: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-05 13:07:17.988062: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-05 13:07:17.988079: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cdd2a59992ee): /proc/driver/nvidia/version does not exist
2023-07-05 13:07:17.988225: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


391/391 [==============================] - 2s 4ms/step - loss: 16631.4980 - mae: 36.2964 - mse: 16631.4824
Epoch 2/30
391/391 [==============================] - 1s 3ms/step - loss: 12524.4258 - mae: 30.4458 - mse: 12524.4150
Epoch 3/30
391/391 [==============================] - 1s 3ms/step - loss: 7763.9531 - mae: 22.9215 - mse: 7763.9458
Epoch 4/30
391/391 [==============================] - 1s 4ms/step - loss: 4764.7900 - mae: 15.9934 - mse: 4764.7793
Epoch 5/30
391/391 [==============================] - 1s 3ms/step - loss: 3026.2671 - mae: 11.0765 - mse: 3026.2556
Epoch 6/30
391/391 [==============================] - 1s 3ms/step - loss: 1897.5980 - mae: 8.4069 - mse: 1897.5858
Epoch 7/30
391/391 [==============================] - 1s 3ms/step - loss: 1216.2369 - mae: 7.0488 - mse: 1216.2251
Epoch 8/30
391/391 [==============================] - 1s 3ms/step - loss: 806.0587 - mae: 6.3107 - mse: 806.0458
Epoch 9/30
391/391 [==============================] - 1s 3ms/step - loss: 611.1533 -

In [7]:
import numpy as np
pred_X  = autoencoder.predict(X)
mse_X = np.mean(np.power(X-pred_X,2), axis=1)
mae_X = np.mean(np.abs(X-pred_X), axis=1)
datadf['mse_X'] = mse_X
datadf['mae_X'] = mae_X

In [8]:
n = 1000
df =  datadf.sort_values(by='mae_X',ascending=False)
df = df.head(n)
rate = df[df['is_attribute']==1].shape[0]/n
print('Top{}的准确率为:{}'.format(n,1-rate))

Top1000的准确率为:0.988


I0706 12:05:03.158010  4015 zk_client.cc:58] node watcher with event type 3, state 3
I0706 12:05:03.394479  4015 zk_client.cc:58] node watcher with event type 3, state 3
